In [64]:
import os
import string
import re
import math
import statistics
from math import sqrt
import numpy as np
import random
import time
import pandas as pd
import nltk, re, pprint
#nltk.download('punkt')
from nltk import word_tokenize
random.seed(1)
np.random.seed(1)
import matplotlib.pyplot as plt
from IPython.display import clear_output
from matplotlib import pyplot as plt
import collections
%matplotlib inline

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.autograd as autograd

import pickle

os.chdir('C:/Users/torna/Documents/StatApp/StatApp')
#os.chdir('/Users/alainquartierlatente/Desktop/Ensae/StatApp')
#os.chdir('/home/aqlt/Documents/Ensae/StatApp')
#os.chdir('C:/Users/Kim Antunez/Documents/Projets_autres/StatApp')

# Sur 100K

In [3]:
nom_dossier = "100k"
# Penser à changer selon taille
with open("data/corpus_trie%s.file" %nom_dossier, "rb") as f:
    corpus = pickle.load(f) 
ens_tweets = [phrase.split() for phrase in corpus]
phrases = ens_tweets
print(len(phrases))

99718


In [6]:
phrases[0:3]

[['il',
  'mérite',
  'd',
  'être',
  'bloquer',
  'la',
  'lettre',
  'de',
  'l',
  'alphabet'],
 ['nickname',
  'et',
  'fière',
  'je',
  't',
  'en',
  'voi',
  'att',
  'j',
  'avais',
  'oublié'],
 ['il', 'est', '1', 'heure']]

In [63]:
from gensim.models import word2vec
model = word2vec.Word2Vec(phrases, size=20, window=3,negative=5,alpha=0.01,seed=1,
                          min_count=5, workers=4, iter=10)
# phrases = les phrases du corpus
# size = dimension du vecteur
# window = fenetre
# negative = nb de neg samples
# alpha = learning rate
# seed 
# min_count = fréquence min des mots qui nous intéresse
# workers = nb de cores sur l'ordi
# iter = nb epoch
model.corpus_count

KeyboardInterrupt: 

In [52]:
vocab = model.wv.vocab
list(vocab)[:5]

['il', 'mérite', 'd', 'être', 'bloquer']

In [53]:
model.wv['wesh']

array([-1.0212375 ,  0.43549496,  2.3465245 , -1.2005298 ,  1.1559012 ,
        0.057466  , -0.10714477,  0.6737687 , -0.7785982 , -0.24978723,
        2.3176663 ,  0.23717044, -1.169239  , -0.21243164, -0.8299047 ,
        1.730058  ,  0.9359951 , -2.3995419 ,  1.5362434 , -0.19903699],
      dtype=float32)

In [54]:
model.similarity("homme","femme")

C:\Users\torna\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).
  """Entry point for launching an IPython kernel.


0.582294

In [65]:
def cos_distance(u, v):
    return (np.dot(u, v)  / (math.sqrt(np.dot(u, u)) *  (math.sqrt(np.dot(v, v)))))

def eucl_distance(u, v):
    return (np.linalg.norm(u-v))

In [66]:
def distance_mots(word1,word2):
    if word1 in vocab and word2 in vocab:
        word_distance = model.similarity(word1,word2)
    else:
         word_distance = float('nan')
    return word_distance
distance_mots_v = np.vectorize(distance_mots)

In [57]:
distance_mots("homme","femme")

C:\Users\torna\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).
  This is separate from the ipykernel package so we can avoid doing imports until


0.582294

In [58]:
df_base = pd.read_csv('data_bis/word_similarity.csv', sep=";")
df_base

,word1,word2,corr
0,corde,sourire,0.00
1,midi,ficelle,0.00
2,coq,périple,0.06
3,fruit,fournaise,0.11
4,autographe,rivage,0.00
...,...,...,...
60,coussin,oreiller,3.00
61,cimetière,cimetière,4.00
62,automobile,auto,3.94
63,joyau,bijou,3.22


In [59]:
distance_mots_v = np.vectorize(distance_mots)
df = df_base
df["corr_word2vec_cos"] = distance_mots_v(df["word1"],df["word2"])
print(len(df))
df = df.dropna()
print(len(df))
df

65
13


C:\Users\torna\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).
  This is separate from the ipykernel package so we can avoid doing imports until


,word1,word2,corr,corr_word2vec_cos
0,corde,sourire,0.00,0.206890
16,côte,forêt,0.22,0.726424
20,garçon,sage,0.29,-0.000511
23,gars,sorcier,0.44,-0.013237
28,oiseau,bois,0.06,0.343423
33,auto,voyage,0.33,0.017356
35,verre,bijou,0.56,0.499181
38,frère,gars,2.00,0.683955
39,sage,sorcier,0.83,0.239894
43,nourriture,fruit,2.78,0.659608


In [60]:
from scipy.stats import spearmanr
#On fait des tests à 5 % pour la distance cosinus
alpha = 0.05
corr, p_value = spearmanr(df["corr"], df["corr_word2vec_cos"])
if p_value > alpha:
    print('Le résultat de word2vec COSINUS est différent de celui du human judgement (non rejet de H0 = non corrélation) p=%.3f' % p_value,'/ Valeur de la corrélation : %.3f'% corr)
else:
    print('Le résultat de word2vec COSINUS est semblable celui du human judgement (rejet de H0 = non corrélation) p=%.3f' % p_value,'/ Valeur de la corrélation : %.3f'% corr)

print("\n")

Le résultat de word2vec COSINUS est différent de celui du human judgement (non rejet de H0 = non corrélation) p=0.280 / Valeur de la corrélation : 0.324




# Sur l'ensemble

In [67]:
nom_dossier = "ens"
# Penser à changer selon taille
with open("data/corpus_trie%s.file" %nom_dossier, "rb") as f:
    corpus = pickle.load(f) 
ens_tweets = [phrase.split() for phrase in corpus]
phrases = ens_tweets
print(len(phrases))

1348627


In [72]:
model = word2vec.Word2Vec(phrases, size=100, window=3,negative=5,alpha=0.01,seed=1,
                          min_count=5, workers=4, iter=100)
# phrases = les phrases du corpus
# size = dimension du vecteur
# window = fenetre
# negative = nb de neg samples
# alpha = learning rate
# seed 
# min_count = fréquence min des mots qui nous intéresse
# workers = nb de cores sur l'ordi
# iter = nb epoch
model.corpus_count

1348627

In [73]:
vocab = model.wv.vocab
list(vocab)[:5]

['il', 'mérite', 'd', 'être', 'bloquer']

In [74]:
df_base = pd.read_csv('data_bis/word_similarity.csv', sep=";")
distance_mots_v = np.vectorize(distance_mots)
df = df_base
df["corr_word2vec_cos"] = distance_mots_v(df["word1"],df["word2"])
print(len(df))
df = df.dropna()
print(len(df))
df

65
52


C:\Users\torna\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).
  This is separate from the ipykernel package so we can avoid doing imports until


,word1,word2,corr,corr_word2vec_cos
0,corde,sourire,0.00,0.142178
1,midi,ficelle,0.00,-0.031496
2,coq,périple,0.06,0.286557
5,automobile,sorcier,0.00,0.041289
7,grimace,instrument,0.00,0.116685
8,refuge,fruit,0.00,0.372516
9,refuge,moine,0.22,0.333038
10,cimetière,asylum,0.22,0.134455
11,garçon,coq,0.44,0.444427
12,verre,magicien,0.06,0.259648


In [75]:
from scipy.stats import spearmanr
#On fait des tests à 5 % pour la distance cosinus
alpha = 0.05
corr, p_value = spearmanr(df["corr"], df["corr_word2vec_cos"])
if p_value > alpha:
    print('Le résultat de word2vec COSINUS est différent de celui du human judgement (non rejet de H0 (= non corrélation) p=%.3f' % p_value,'/ Valeur de la corrélation : %.3f'% corr)
else:
    print('Le résultat de word2vec COSINUS est semblable celui du human judgement (rejet de H0 (= non corrélation)) p=%.3f' % p_value,'/ Valeur de la corrélation : %.3f'% corr)

print("\n")

Le résultat de word2vec COSINUS est semblable celui du human judgement (rejet de H0 = non corrélation) p=0.000 / Valeur de la corrélation : 0.479


